In [0]:
# ==========================================================
# UDV - HM_TABLAS_CLASIFICACION
# Proyecto: Liga 1 Perú
# Autor: Oscar García Del Águila
# ==========================================================

from env_setup import *
from utils_liga1 import (
    setup_adls, get_dbutils, get_predecesor, get_pipeline_params,
    get_yaml_from_param, read_udv_table, get_pipeline,
    write_delta_udv, log, is_dataframe_empty, get_entity_data
)
from pyspark.sql import SparkSession
from hm_tablas_clasificacion import  carga_final_hm_tablas_clasificacion
import traceback


# ----------------------------------------------------------
# CONFIG BASICA
# ----------------------------------------------------------
log("Inicio pipeline HM_TABLAS_CLASIFICACION", "INFO","hm_tablas_clasificacion")

spark = SparkSession.builder.getOrCreate()
setup_adls()
dbutils = get_dbutils()

In [0]:
# ----------------------------------------------------------
# PARÁMETROS Y PREDECESORES
# ----------------------------------------------------------
try:
    # Para pruebas en notebook; en ADF se setea dinámicamente
    dbutils.widgets.text("prm_pipelineid", "20")
    prm_pipelineid = int(dbutils.widgets.get("prm_pipelineid"))

    pipeline_info    = get_pipeline(prm_pipelineid)
    entity_name      = pipeline_info["pipeline"]          # 'hm_tablas_clasificacion'

    dict_predecesores = get_predecesor(prm_pipelineid)
    dict_params       = get_pipeline_params(prm_pipelineid)

    # Predecesores:
    #  - md_catalogo_equipos     -> RutaTabla = 'tb_udv.md_catalogo_equipos'
    #  - tablas_clasificacion    -> RutaTabla = 'tablas_clasificacion' (RAW)
    prm_tabla_md_catalogo_equipos = dict_predecesores["md_catalogo_equipos"]["RutaTabla"]
    entity_raw_clasificacion      = dict_predecesores["tablas_clasificacion"]["RutaTabla"]

    # Parámetros UDV
    prm_capa_udv     = dict_params["CAPA_UDV"]
    prm_ruta_base    = dict_params["RUTA_BASE"]
    prm_ruta_tabla   = dict_params["RUTA_TABLA"]
    prm_formato      = dict_params["FORMATO_SALIDA"]
    prm_schema_tb    = dict_params["SCHEMA_TABLA"]
    prm_tabla_output = dict_params["NOMBRE_TABLA"]
    prm_ruta_yaml    = dict_params["YAML_PATH"]

    log("Parámetros cargados correctamente", "INFO", entity_name)

except Exception as e:
    log(f"Error al cargar parámetros o predecesores: {e}", "ERROR", entity_name)
    print(traceback.format_exc())
    raise

In [0]:
# ----------------------------------------------------------
# LECTURA YAML
# ----------------------------------------------------------
try:
    yaml_conf = get_yaml_from_param(prm_ruta_yaml)

    prm_campo_json                 = yaml_conf[entity_name]["campo_json"]
    prm_cols_raw_clasificacion_hm  = yaml_conf[entity_name]["cols_raw_clasificacion_hm"]
    prm_rename_columns_hm          = yaml_conf[entity_name]["rename_columns_hm"]
    prm_schema_hm                  = yaml_conf[entity_name]["schema_hm"]

    log("YAML cargado correctamente", "INFO", entity_name)

except Exception as e:
    log(f"Error al leer YAML {prm_ruta_yaml}: {e}", "ERROR", entity_name)
    print(traceback.format_exc())
    raise

In [0]:
# ----------------------------------------------------------
# LECTURA PREDECESORES (MD_CATALOGO_EQUIPOS + RAW JSON)
# ----------------------------------------------------------
try:
    log("Lectura de predecesores", "INFO", entity_name)

    # 1) MD_CATALOGO_EQUIPOS
    df_md_catalogo_equipos = read_udv_table(prm_tabla_md_catalogo_equipos)
    if is_dataframe_empty(df_md_catalogo_equipos):
        raise Exception(f"No se encontró data en: {prm_tabla_md_catalogo_equipos}")

    # 2) RAW tablas_clasificacion desde tbl_paths (flg_udv = 'N') + JSON
    df_raw = get_entity_data(entity_raw_clasificacion)
    if is_dataframe_empty(df_raw):
        raise Exception("RAW vacío")

    log("Predecesores completados correctamente", "INFO", entity_name)


except Exception as e:
    log(f"Error en lectura de predecesores/RAW: {e}", "ERROR", entity_name)
    print(traceback.format_exc())
    raise

In [0]:
# ----------------------------------------------------------
# EJECUCIÓN PRINCIPAL
# ----------------------------------------------------------
try:
    log("Inicio de Ejecución Principal", "INFO", entity_name)

    df_final_hm = carga_final_hm_tablas_clasificacion(
    df_raw=df_raw,
    df_md_catalogo_equipos=df_md_catalogo_equipos,
    prm_cols_raw_clasificacion_hm=prm_cols_raw_clasificacion_hm,
    prm_rename_columns_hm=prm_rename_columns_hm,
    prm_schema_hm=prm_schema_hm,
    campo_json=prm_campo_json
    )


    if is_dataframe_empty(df_final_hm):
        log("df_final_hm vacío, no se realizará escritura en UDV", "WARN", entity_name)
    else:
        log("Escribiendo HM_TABLAS_CLASIFICACION con overwrite dinámico por partición [periodo]", "INFO", entity_name)

        write_delta_udv(
            spark,
            df_final_hm,
            prm_schema_tb,          # ej: 'tb_udv'
            prm_tabla_output,       # ej: 'hm_tablas_clasificacion'
            mode="overwrite",
            partition_by=["periodo"],
            overwrite_dynamic_partition=True
        )

        log("Escritura HM_TABLAS_CLASIFICACION completada", "SUCCESS", entity_name)


except Exception as e:
    log(f"Error en ejecución principal: {e}", "ERROR", entity_name)
    print(traceback.format_exc())
    raise

log("Finalización del pipeline UDV", "INFO", entity_name)